In [1]:
import pandas as pd
import datetime as dt
from datetime import timedelta

In [2]:
def config():
    global options, coerce_column_to_numeric, dir_path_saved
    options = pd.options.mode.chained_assignment = None
    def coerce_column_to_numeric(df,column):
        df[column] = df[column].apply(pd.to_numeric, errors='coerce')
    dir_path_saved = r'../database_dashboard/'
config()

In [5]:
def load_data_marts():
    global real_state_date_rank, real_state_rank
    real_state_rank = pd.read_parquet(r"../database_dashboard/data_mart_all_real_state.parquet")
    real_state_date_rank = pd.read_parquet(r"../database_dashboard/data_mart_real_state_date.parquet")
load_data_marts()

In [10]:
def ranking_10():
    global fii_rank, fii_rank_value_20, real_state_rank
    options
    fii_rank = real_state_rank[['codigo_do_fundo','setor','preco_atual','dividendo','dividend_yield']]
    coerce_column_to_numeric(fii_rank,'preco_atual')
    coerce_column_to_numeric(fii_rank,'dividend_yield')
    fii_rank['preco_atual'] = fii_rank['preco_atual'].fillna(0)
    fii_rank['dividend_yield'] = fii_rank['dividend_yield'].fillna(0)
    fii_rank['dividend_yield'] = fii_rank['dividend_yield'].astype('float64')
    fii_rank['dividendo'] = fii_rank['dividendo'].astype('float64')
    fii_rank = fii_rank.sort_values(['dividend_yield','dividendo','preco_atual'],ascending=False)
    fii_rank_value = fii_rank.loc[fii_rank['preco_atual'] < 100]
    fii_rank_value = fii_rank_value.sort_values(['dividend_yield'],ascending=False)
    fii_rank_value_20 = fii_rank_value.head(20)
ranking_10()

In [11]:
fii_rank_value_20.sort_values('codigo_do_fundo')

,codigo_do_fundo,setor,preco_atual,dividendo,dividend_yield
116,AFHI11,Títulos e Val Mob,99.58,1.35,1.31
16,AIEC11,Lajes Corporativas,80.91,1.31,1.57
7,ARRI11,Títulos e Val Mob,96.85,1.34,1.35
42,BIME11,,9.16,0.12,1.30
61,DEVA11,Títulos e Val Mob,96.15,1.31,1.31
85,EQIN11,Títulos e Val Mob,93.50,1.33,1.41
66,FLCR11,Títulos e Val Mob,98.00,1.34,1.34
102,GCRI11,Títulos e Val Mob,98.70,1.48,1.49
77,IBCR11,Títulos e Val Mob,96.00,1.47,1.51
245,KINP11,Residencial,12.80,0.30,2.35


In [52]:
only_date_real_state = real_state_date_rank[['codigo_do_fundo','data_base','data_de_pagamento']]

In [47]:
hj = dt.datetime.today()
dia = hj.day -1
month_init = hj - timedelta(days=dia)

In [54]:
only_date_real_state.loc[only_date_real_state['codigo_do_fundo'] == 'BIME11']

,codigo_do_fundo,data_base,data_de_pagamento


In [50]:
only_date_real_state = only_date_real_state.loc[only_date_real_state['data_de_pagamento'] > month_init]

In [56]:
df_final = pd.merge(fii_rank_value_20,only_date_real_state)

In [57]:
df_final['data_limite_de_compra'] = df_final['data_base'] - timedelta(days=2)

In [59]:
df_final['preco_atual'] = df_final['preco_atual'].fillna(0)
df_final['dividendo'] = df_final['dividendo'].fillna(0) 
df_final['dividend_yield']= df_final['dividend_yield'].fillna(0)
#df_final['tipo'] = df_final['tipo'].fillna('Nao Identificado')          

In [61]:
df_final.to_parquet(r'..\database_dashboard\data_mart_top_20_real_state_funds.parquet',index=False)